<a href="https://colab.research.google.com/github/toshimasay/AIciaSolidProject/blob/master/Tinder_K_medoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
、!pip install scikit-learn-extra

     |████████████████████████████████| 624kB 2.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.1.0b2-cp36-cp36m-linux_x86_64.whl size=340329 sha256=f851992961627f8aefcaeb993dd477b93358dc4ce3d08f256ab4e2929a1fe853
  Stored in directory: /root/.cache/pip/wheels/04/01/0f/943bffb48bac048fa216b4325f1a6c939491ccb0ff500e08f4
Successfully built scikit-learn-extra


In [ ]:
%%time
#1:マウントしたドライブのIDを取得（URLなどからコピーしてくる）
import os
folder_id = "1g1VskqTzazHY7eWPtYLFYyPZXrlpZWB4"
os.environ['GOOGLE_DRIVE_FOLDER_ID'] = folder_id

# 認証処理に必要なモジュールのインストール
!apt-get install -y -qq software-properties-common module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/nullｌd
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

#2: Google Colab APIのアカウント認証を得る（colabとgoogle driveをつなげる）
from google.colab import auth
auth.authenticate_user()

#3: Driveをマウントするためのアカウント認証を得る（Driveの変更権限など）
## 自分のGoogleアカウントに割り振られているトークンの取得
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

## google-drive-ocamlfuseを利用してdriveをマウントするための認証を得る
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#4: 特定のフォルダをマウント（/content/drive/以下をMyDriveから任意のフォルダに上書きする）
!cp -f ~/.gdfuse/default/config config
!sed -i -e "s/^root_folder=$/root_folder=${GOOGLE_DRIVE_FOLDER_ID}/" config
!mkdir -p drive
!google-drive-ocamlfuse -config ./config -cc drive

#5: マウントしたフォルダに移動
%cd /content/drive/

Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&ac

In [ ]:
%ls

 あ
 テンプレコード/
 ワードクラウド/
 0605_udemy_daily_data.xlsx
 config
 drive/
 アイロボットフォロワーID.csv
 iRobot_Style_follower_ids.csv
 mecab-ipadic-neologd/
 Miniconda3-latest-Linux-x86_64.sh
 models/
 __pycache__/
'pythonCSV&EXCEL'/
 rdkit_installer.py
 results/
 Tinder/
 Tinder_monthly/
 Tinder_ラベル付き.csv
 Tinder_教師データ.txt
 Udemyコード/
 UdemyJapan_follower_ids.csv
 Udemy_フォロワー一覧_8月.xlsx
 使わないcolab/
 ワードクラウド元データ/
 分析コード/
 北海道.csv
 山之内練習用/
 フォロー情報つき　Udemy_クラスター分析メタデータ_ver03.csv
 【構築データ】Udemyレポート_7月.xlsx
 総当たり重回帰テストデータ.xlsx
 ユーザー辞書.xlsx


In [ ]:
import pandas as pd
import numpy as np
import sys
import csv
import json
import time
import re
import os
from datetime import datetime
import pickle
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

In [ ]:
# 分かち書きしたクチコミデータ
f = open('Tinder_教師データ.txt', 'rb')
sentences = pickle.load(f)
f.close()

In [ ]:
len(sentences)

1624

In [ ]:
vectorizer = CountVectorizer(max_df=0.9)

In [ ]:
corpus = [' '.join(wl) for wl in sentences]

In [ ]:
# tfidfベクトルに変換
X = vectorizer.fit_transform(corpus)

In [ ]:
count_vector = X.toarray()

In [ ]:
#ベクトルの素性に使われている単語一覧
words = vectorizer.get_feature_names()

In [ ]:
count_vector_df = pd.DataFrame(count_vector, columns=words)

In [ ]:
count_vector_df

,0kg,0m,0mm,0t,0v,0w,100,100km,100人,10万,10分,11時,122,12人,151cm,152cm,16時,172cm,174cm,18歳,1994年生まれ,19歳,1b,1d,1m,1t,1つ,1チャンネル,1万人,1万円,1人,1位,1回,1年,1度,1日,1時間,1本,1枚,1番,...,飲む,飲める,飲食店,飼う,飽きる,飽く,飾り,餌食,香ばしい,香港,馬場,馬鹿,馬鹿馬鹿しい,駄々,駄目,騙す,驚き,高い,高さ,高まる,高みの見物,高める,高卒,高校,高校時代,高校生,高身長,高額,鬱憤,鬼道,魅力,魔羅,鳥肌,鳴り響く,鸚鵡返し,麻布,黒歴史,黒髪,黙る,鼻炎
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1620,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1622,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
columns  = count_vector_df.columns.to_list()

In [ ]:
#1以上の条件でBooleanに変換した後intにキャスト変換
onehot_df = (count_vector_df>=1.0).astype(int)

In [ ]:
#n_cluster：クラスタ数
#max_iter：最大反復回数
#metrics：距離尺度使えるっぽいのは以下の種類
#[‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’]
#[‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘correlation’, ‘dice’, ‘hamming’, ‘jaccard’, ‘kulsinski’, ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, ‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’]
#init：初期値の調整（k-medoids++：複数回試行して一番クラスタの偏りがない点を選択するので個人的にはこちらを推奨）

from sklearn_extra.cluster import KMedoids
X = np.asarray(onehot_df)
kmedoids = KMedoids(n_clusters=2, max_iter=500, metric='jaccard',init='k-medoids++', random_state=0).fit(X)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [ ]:
#振り分けられたラベルの一覧
kmedoids.labels_

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#各クラスタの中心事象（ツイート）
kmedoids.cluster_centers_

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
center1 = kmedoids.cluster_centers_[0]
center2 = kmedoids.cluster_centers_[1]

In [ ]:
#中心事象に使われている単語一覧
center1_indexes = [i for i,c1 in enumerate(center1) if c1==1]
center1_words = [onehot_df.columns[index] for index in center1_indexes]
center1_words

['くる', 'たくさん', 'わかる', 'わけ', '会う', '写真', '出る', '見る', '言う']

In [ ]:
center2_indexes = [i for i,c1 in enumerate(center2) if c1==1]
center2_words = [onehot_df.columns[index] for index in center2_indexes]
center2_words

['20代', '人間', '問題', '男性', '遊べる']

In [ ]:
#各クラスター内の二乗誤差
#ただし、凸面や等方性を想定しており、細長い集合などイレギュラーな構造が合わさった集合などを扱うのには向いていない。
#また、正規化された指標ではないので「低い方が良く、0が最適」以上のことは言えない

kmedoids.inertia_

1592.1742032784862

In [ ]:
# 試しに1ツイート目のデータでクラスタの予測結果をみる
kmedoids.predict(np.asarray(onehot_df.iloc[:1]))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


array([0])

In [ ]:
datas = pd.read_csv("Tinder_ラベル付き.csv")

In [ ]:
datas["two_cluster"] = kmedoids.labels_

In [ ]:
datas.head()

,Unnamed: 0,created_at,id_str,text,flug1,flug2,score,flug,two_cluster
0,0,2003-06-20 13:11:00,1.24e+18,【YouTube更新！！！】\n\n「ティンダーでヤリモク女子をお持ち帰りしたいなら絶対にや...,2,2,2.0,True,0
1,1,2020-07-22 05:47:00,1.29e+18,マッチングアプリで\nめちゃくちゃ業者に間違えられるんだけど。\n真逆なんだけど。笑\n\n...,0,0,0.0,True,0
2,3,2020-02-29 06:08:00,1.23e+18,やはり\nティンダーは香ばしいですな。,3,3,3.0,True,0
3,5,2004-04-20 13:02:00,1.25e+18,がっつり出勤してる身からすると1日だけ外出する日が増えても結果は変わらない気がするので、有楽...,1,1,1.0,False,0
4,6,2020-07-16 13:22:00,1.28e+18,なんでわいはティンダーで話してる子とまで就活の話をしとるんだ,0,0,0.0,True,0
